In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
import requests


ACCESS API

In [9]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

r = requests.get(url)
json = r.json()


In [3]:
json.keys()


dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [4]:
pd.options.display.max_columns = None

In [73]:
elements_df = pd.DataFrame(json['elements'])

elements_df.columns.sort_values()


Index(['assists', 'bonus', 'bps', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'clean_sheets', 'code',
       'corners_and_indirect_freekicks_order',
       'corners_and_indirect_freekicks_text', 'cost_change_event',
       'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall',
       'creativity', 'creativity_rank', 'creativity_rank_type',
       'direct_freekicks_order', 'direct_freekicks_text', 'dreamteam_count',
       'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name',
       'form', 'goals_conceded', 'goals_scored', 'ict_index', 'ict_index_rank',
       'ict_index_rank_type', 'id', 'in_dreamteam', 'influence',
       'influence_rank', 'influence_rank_type', 'minutes', 'news',
       'news_added', 'now_cost', 'own_goals', 'penalties_missed',
       'penalties_order', 'penalties_saved', 'penalties_text', 'photo',
       'points_per_game', 'red_cards', 'saves', 'second_name',
       'selected_by_percent', 'special', 'squad_

In [30]:
teams_df = pd.DataFrame(json['teams'])
teams_df

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1190,1250,1110,1140,1110,1170,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1130,1160,1110,1120,1130,1160,2
2,94,0,None,3,0,Brentford,0,0,0,BRE,3,None,False,0,1070,1080,1130,1160,1100,1150,130
3,36,0,None,4,0,Brighton,0,0,0,BHA,3,None,False,0,1140,1140,1160,1190,1090,1130,131
4,90,0,None,5,0,Burnley,0,0,0,BUR,2,None,False,0,1040,1050,1040,1070,1060,1100,43
5,8,0,None,6,0,Chelsea,0,0,0,CHE,5,None,False,0,1340,1350,1330,1350,1340,1330,4
6,31,0,None,7,0,Crystal Palace,0,0,0,CRY,3,None,False,0,1080,1110,1110,1160,1110,1140,6
7,11,0,None,8,0,Everton,0,0,0,EVE,3,None,False,0,1140,1190,1120,1140,1150,1210,7
8,13,0,None,9,0,Leicester,0,0,0,LEI,3,None,False,0,1170,1190,1100,1130,1120,1160,26
9,2,0,None,10,0,Leeds,0,0,0,LEE,3,None,False,0,1080,1100,1060,1070,1100,1130,9


***Creating dictionary for code to team conversion***

In [33]:
conv_to_code_dic = {}

#Creates dictionary to convert code into
for item in teams_df.index:
  id = teams_df.iloc[item,3]
  short_name = teams_df.iloc[item,5]
  conv_to_code_dic[id] = short_name

def conv_to_code(item):
  try:
    return conv_to_code_dic[item]
  except KeyError:
    pass

***Accessing fixtures and past history of teams using classes***

In [98]:
new_urls = {}

for x  in set(elements_df['team']):
    team = conv_to_code(x)
    new_urls[team] = f'https://fantasy.premierleague.com/api/element-summary/{x}/'

#Function

class fix_ticker():

  def __init__(self,team,new_urls):
    """Initializes items"""
    self.team = team
    self.new_urls = new_urls
    self.url = self.new_urls[self.team.title()]
    self.r_2 = requests.get(self.url)
    self.json_2 = self.r_2.json()
    

  def get_fix(self):
    """Obtains the future fixtures and past scores of each team based on their url"""
    
    player_fixtures_df = pd.DataFrame(self.json_2['fixtures'])  
    player_fixtures_df['away_team'] = player_fixtures_df['team_a'].map(conv_to_code_dic)
    player_fixtures_df['home_team'] = player_fixtures_df['team_h'].map(conv_to_code_dic)
    player_fix_df = player_fixtures_df[['event','home_team','away_team']]
    return player_fix_df

  def get_past_scores(self):
    """Obtains the past scoreline of each team"""
    history_df = pd.DataFrame(self.json_2['history'])
    history_df['opponent'] = history_df['opponent_team'].map(conv_to_code)
    history_trim_df = history_df[['round', 'was_home', 'team_h_score', 'team_a_score', 'opponent']]
    return history_trim_df

In [101]:
new_urls

{'Arsenal': 'https://fantasy.premierleague.com/api/element-summary/1/',
 'Aston Villa': 'https://fantasy.premierleague.com/api/element-summary/2/',
 'Brentford': 'https://fantasy.premierleague.com/api/element-summary/3/',
 'Brighton': 'https://fantasy.premierleague.com/api/element-summary/4/',
 'Burnley': 'https://fantasy.premierleague.com/api/element-summary/5/',
 'Chelsea': 'https://fantasy.premierleague.com/api/element-summary/6/',
 'Crystal Palace': 'https://fantasy.premierleague.com/api/element-summary/7/',
 'Everton': 'https://fantasy.premierleague.com/api/element-summary/8/',
 'Leeds': 'https://fantasy.premierleague.com/api/element-summary/10/',
 'Leicester': 'https://fantasy.premierleague.com/api/element-summary/9/',
 'Liverpool': 'https://fantasy.premierleague.com/api/element-summary/11/',
 'Man City': 'https://fantasy.premierleague.com/api/element-summary/12/',
 'Man Utd': 'https://fantasy.premierleague.com/api/element-summary/13/',
 'Newcastle': 'https://fantasy.premierleagu

In [105]:
burnley_fix = fix_ticker('chelsea',new_urls)
burnley_fix.get_fix()

,event,home_team,away_team
0,24.0,Wolves,Arsenal
1,26.0,Arsenal,Brentford
2,28.0,Watford,Arsenal
3,29.0,Arsenal,Leicester
4,30.0,Aston Villa,Arsenal
5,31.0,Crystal Palace,Arsenal
6,32.0,Arsenal,Brighton
7,33.0,Southampton,Arsenal
8,34.0,Arsenal,Man Utd
9,35.0,West Ham,Arsenal


In [100]:
burnley_fix.get_past_scores()

,round,was_home,team_h_score,team_a_score,opponent
0,1,False,2,0,Brentford
1,2,True,0,2,Chelsea
2,3,False,5,0,Man City
3,4,True,1,0,Norwich
4,5,False,0,1,Burnley
5,6,True,3,1,Spurs
6,7,False,0,0,Brighton
7,8,True,2,2,Crystal Palace
8,9,True,3,1,Aston Villa
9,10,False,0,2,Leicester
